In [ ]:
!pip install "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install "../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"

In [ ]:
!pip install "../input/hpapytorchzoozip/pytorch_zoo-master"

In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

NUC_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
CELL_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device='cuda',
    padding=False,
    multi_channel_model=True
)

In [ ]:
channel_maps = {"red":"microtubule ", 
                "blue":"nuclei", 
                "yellow":"Endoplasmic Reticulum",
                "green":"protein of interest"}

def build_image_names(image_id: str) -> list:
    # mt is the mitchondria
    mt = f'/kaggle/input/hpa-single-cell-image-classification/test/{image_id}_red.png'
    
    # er is the endoplasmic reticulum
    er = f'/kaggle/input/hpa-single-cell-image-classification/test/{image_id}_yellow.png'
    
    # nu is the nuclei
    nu = f'/kaggle/input/hpa-single-cell-image-classification/test/{image_id}_blue.png'
    
    return [mt], [er], [nu], [[mt], [er], [nu]]

mt, er, nu, images = build_image_names(image_id='0040581b-f1f2-4fbe-b043-b6bfea5404bb')

In [ ]:
# For nuclei
nuc_segmentations = segmentator.pred_nuclei(images[2])

# For full cells
cell_segmentations = segmentator.pred_cells(images)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,3,figsize=(40,100))
raw = plt.imread(mt[0])
ax[0].imshow(raw)
ax[1].imshow(cell_segmentations[0])
ax[2].imshow(nuc_segmentations[0])

In [ ]:
# post-processing
i = 0
nuclei_mask, cell_mask = label_cell(nuc_segmentations[i], cell_segmentations[i])

In [ ]:
import numpy as np

i = 0

microtubule = plt.imread(mt[i])    
endoplasmicrec = plt.imread(er[i])    
nuclei = plt.imread(nu[i])
mask = cell_mask
img = np.dstack((microtubule, endoplasmicrec, nuclei))

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.imshow(mask, alpha=0.5)
plt.show()

In [ ]:
np.savez("0040581b-f1f2-4fbe-b043-b6bfea5404bb.npz",mask)

In [ ]:
!rm ./0040581b-f1f2-4fbe-b043-b6bfea5404bb.npz